## Motivation

In [2]:
from astropy.io import ascii
import os 
from gatspy import datasets, periodic
import gatspy

import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = "retina"
from matplotlib import rcParams
rcParams['savefig.dpi'] = 550
rcParams['font.size'] = 20